In [1]:
import re
import unicodedata
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from queue import Queue
import pandas as pd
from indicnlp.tokenize.indic_tokenize import trivial_tokenize_indic

In [ ]:
trivial_tokenize_indic()

In [2]:
class Preprocess:
        
        # --------------------------------------- Constructor --------------------------------------- 
        
        def __init__(self,stopword_list):
            self.data_path = ''
            self.stopword_list = stopword_list
                

        # --------------------------------------- Preprocess --------------------------------------- 
        
        def expand_concatenations(self, word):
            
            
            if not re.match('[a-zA-Z]+', word) or re.match('/d+',word):
                for i in range(len(word)):
                    if not('DEVANAGARI ' in unicodedata.name(word[i])):
                        word = word[:i] if( len(word[i:]) < 2 and not word[i:].isnumeric()) else word[:i] + " " + word[i:]
                        break
            else:
                for i in range(len(word)):
                    if ('DEVANAGARI ' in unicodedata.name(word[i])):
                        word = word[i:] if( len(word[:i]) < 2 and not word[:i].isnumeric() ) else word[:i] + " " + word[i:]
                        break

            return(word)
    
        
        def clean_text(self,text: str) -> str:
            try:
                special_chars = r'''!()-[]{};:'"\,<>./?@#$%^&*_~'''
                stemmer = PorterStemmer()
                lemmatizer = WordNetLemmatizer()

                if not(isinstance(text, str)): text = str(text)

                #Removing unprintable characters
                text = ''.join(x for x in text if x.isprintable())

                # Cleaning the urls
                text = re.sub(r'https?://\S+|www\.\S+', '', text)

                # Cleaning the html elements
                text = re.sub(r'<.*?>', '', text)

                # Removing the punctuations
                text = re.sub('[!#?,.:";-@#$%^&*_~<>()/\-]', '', text)


                # Removing stop words
                text = ' '.join([word for word in text.split() if word not in self.stopword_list])

                # Expanding noisy concatenations (Eg: algorithmआणि  -> algorithm आणि ) 
                text = ' '.join([self.expand_concatenations(word) for word in text.split()])

#                 preprocessed_text = ""

#                 for word in text.split(): 
#                     if (re.match('\d+', word)):
#                         if(word.isnumeric()):
#                             preprocessed_text = preprocessed_text + '#N' + " "
#                         else:
#                             preprocessed_text = preprocessed_text + word.lower() + " "

#                     else:
#                         if(re.match('[a-zA-Z]+', word)):
#                             if not len(word) < 2:
#                                 word = word.lower()
#     #                             word = lemmatizer.lemmatize(word, pos='v')
#                                 preprocessed_text = preprocessed_text + word + " "

#                         else:
#                             preprocessed_text = preprocessed_text + word + " "

#                 return preprocessed_text
                return text
            
            except ValueError as ve:
                print('Error processing:\t',text)
                return ''
    
        def preprocess_text(self,text: str) -> str:

            try:
                if not(isinstance(text, str)): text = str(text)
                preprocessed_text = ""

                for word in text.split(): 
                    if (re.match('\d+', word)):
                        if(word.isnumeric()):
                            preprocessed_text = preprocessed_text + '#N' + " "
                        else:
                            preprocessed_text = preprocessed_text + word.lower() + " "

                    else:
                        if(re.match('[a-zA-Z]+', word)):
                            if not len(word) < 2:
                                word = word.lower()
    #                             word = lemmatizer.lemmatize(word, pos='v')
                                preprocessed_text = preprocessed_text + word + " "

                        else:
                            preprocessed_text = preprocessed_text + word + " "

                return preprocessed_text

            except ValueError as ve:
#                 print('Error processing:\t',text)
                return ''
            
        def split_devanagri_word(self,word: str, punctuations = True) -> str:
            try:
                q = Queue()
                l_index = 0
                if not(isinstance(word, str)): word = str(word)
                tokens = []
                
                for char in word:
                    
#                     print(char, '--->', unicodedata.name(char))
                    if not 'devanagari' in unicodedata.name(char).lower():
                        tokens.append(char)
                        continue
                        
                    if 'letter' in unicodedata.name(char).lower():
                        if q.empty():
                            tokens.append(char)
                        else:
                            while not q.empty():
                                tokens[len(tokens)-1] += q.get() 
                            tokens.append(char)   
                    else:
                        if punctuations == True:
                            q.put(char)
                    
                for i, char in reversed(list(enumerate(tokens.copy()))):
                    if('devanagari' in unicodedata.name(char).lower()):
                        l_index = i
#                         print(l_index)
                        break
                        
                while not q.empty():
                        tokens[l_index] += q.get() 
                
                return tokens
                
            except Exception as e:
#                 print('Error processing:\t',word)
                return ''
        
        def text2characters(self,text:str, punctuations = True)->str:
            try:
                if not(isinstance(text, str)): text = str(text)
                char_sequence = ""
                char_list = []
                
                for word in text.split():
                    seq = ' '.join([char for char in self.split_devanagri_word(word, punctuations)])                
                    char_sequence = char_sequence + seq + ' '
    
#                     print(word,'--->',seq)
                    
                return char_sequence
            
            except ValueError as ve:
                print('Error processing:\t',text)
                return ''
            
            
        def tokenize_characters(self, document):
            vocab = set()
            cnt = 0
            token_dict = {}
            
            if isinstance(document, list):
#                 print('Doc')
                for text in document:
                    char_sequence = self.text2characters(text)
                    tokens_indic = pd.Series(trivial_tokenize_indic(char_sequence))
                    word_counts = tokens_indic.value_counts()
                    
                    vocab = vocab.union(set(word_counts.keys()))

                print('Total Unique Tokens (Characters): {}'.format(len(vocab)))

                for char in vocab:
                    cnt += 1
                    token_dict[char] = cnt
            
            else:
#                 print('sent')
                char_sequence = self.text2characters(document)
                tokens_indic = pd.Series(trivial_tokenize_indic(char_sequence))
                word_counts = tokens_indic.value_counts()  
                vocab = vocab.union(set(word_counts.keys()))

                print('Total Unique Tokens (Characters): {}'.format(len(vocab)))

                for char in vocab:
                    cnt += 1
                    token_dict[char] = cnt
                
            return token_dict

        
        def text_to_sequence(self,document,token_dict):
            
            sequence_doc = []
            if isinstance(document, list):
                print('Total records: ',len(document))
                cnt = 0
                for text in document:
                    try:
                        char_array = self.text2characters(text).split()
                        text_sequence = [token_dict[x] for x in char_array]
                        sequence_doc.append(text_sequence)
                        cnt+=1
                    except:
                        print(text)
                        
                print('Records converted: ',cnt)
                
            else:
                char_array = self.text2characters(document).split()
                text_sequence = [token_dict[x] for x in char_array]
                sequence_doc.append(text_sequence)
                print('Records converted: 1')
                
            return sequence_doc
            

In [80]:
if __name__ == '__main__':
    df = pd.read_csv('../Technodifacation/Data/training_data_marathi.csv')
    
    sampletext1 = df['text'].sample().values
    print(sampletext1)
    pp = Preprocess([])
    sampletext2 = 'त्यांना जनतेला पटवून द्यावे लागेल99'

    test_list1 = ['त्यांना','H20', '2H20','Animal2Animal' ,'सी२ओ२', 'लागेल99', 'Animalत्यांना',
                 'त्यांनाAnimal', 'Analogy_त्यांना', 'Science२१', '१२Number', '!@)$&%!#)&$!&$!$B Bo ', '११.२२','I', '१','1','11.22','a','B','सी']

    test_list2 = ['त्यांना CO2 2H20 सीओ२ लागेल99 , Animalत्यांना त्यांनाAnimal Analogy_त्यांना Science२१ १२Number',
                 '!@)$&%!#)&$!&$!$I am Atharva ११.२२ Kulkarni 11.22 a B 1 सी']

    for text in test_list2:
        print(text, '\t--->\t', pp.clean_text(text),'\n')   

['तर , इंटरप्ट डिस्क्रिप्टर टेबलची सामग्री काय आहे ?']
त्यांना CO2 2H20 सीओ२ लागेल99 , Animalत्यांना त्यांनाAnimal Analogy_त्यांना Science२१ १२Number 	--->	 त्यांना CO2  2H20 सीओ२ लागेल 99 Animal त्यांना त्यांना Animal Analogy त्यांना Science २१ १२ Number 

!@)$&%!#)&$!&$!$I am Atharva ११.२२ Kulkarni 11.22 a B 1 सी 	--->	 I am Atharva ११२२ Kulkarni  1122 a B  1 सी 



In [3]:
pp = Preprocess([])
from indicnlp.syllable import  syllabifier

In [4]:
sample_word =  'जगदीशचंद्र'
tokens = pp.split_devanagri_word(sample_word, punctuations=True)
tokens

['ज', 'ग', 'दी', 'श', 'चं', 'द्', 'र']

In [5]:
text = 'काही संस्थांना  10/3/2017 लिहण्याची प्रथा आहे'

clean_text = pp.clean_text(text)

char_sequence_1 = pp.text2characters(clean_text)
char_sequence_2 = pp.text2characters(clean_text, punctuations=False)
print('\nText: ',clean_text,'\n\nWith Punctuations: ',char_sequence_1,'\n\nOnly Letters: ',char_sequence_2)


Text:  काही संस्थांना  1032017 लिहण्याची प्रथा आहे 

With Punctuations:  का ही सं स् थां ना 1 0 3 2 0 1 7 लि ह ण् या ची प् र था आ हे  

Only Letters:  क ह स स थ न 1 0 3 2 0 1 7 ल ह ण य च प र थ आ ह 


In [7]:
df = pd.read_csv('../dataset/original-dataset/marathi-training-data.csv')
df

,text,label
0,"प्रा . प्रताप हरिदास : होय , मला वाटते की हा ए...",com_tech
1,"तर , विशिष्ट गोष्टींद्वारे , ठराविक कायद्यांद्...",bioche
2,- - - - - - - - - - - - - - - - - - - - - - - ...,cse
3,"तर , आपला अर्धा चिन्ह 9 वाजता असेल .",phy
4,"म्हणून , मी असे म्हणालो की जर शेकडो , हजारो कि...",phy
...,...,...
41992,"जरी आपण डेटा कूटबद्ध केला , तरीही हा मुख्य व्य...",cse
41993,"ते म्हणतात - "" ज्याला पाहण्यासाठी डोळे , ऎकण्य...",com_tech
41994,"प्रथम क्रोनोलॉजिकल , क्रॉनोलॉजी म्हणजे आपल्याल...",com_tech
41995,"त्या थोड्या तपशीलावर येईल , जेणेकरून संपूर्ण ग...",bioche


In [12]:
text = "जरी आपण डेटा कूटबद्ध केला ani99."
#clean_text = pp.clean_text(text)

char_sequence_1 = pp.text2characters(pp.clean_text(text))
char_sequence_2 = pp.text2characters(pp.clean_text(text), punctuations=False)
print('\nText: ',text,'\n\nWith Punctuations: ',char_sequence_1,'\n\nOnly Letters: ',char_sequence_2)


Text:  जरी आपण डेटा कूटबद्ध केला ani99. 

With Punctuations:  ज री आ प ण डे टा कू ट ब द् ध के ला a n i 9 9  

Only Letters:  ज र आ प ण ड ट क ट ब द ध क ल a n i 9 9 


# Idiotic Keras

In [16]:
import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level = False, split = " ")
tokenizer.fit_on_texts(char_sequence_1)
print(tokenizer.word_counts)

OrderedDict([('प', 11), ('्', 19), ('र', 14), ('ा', 26), ('त', 12), ('ह', 7), ('ि', 11), ('द', 5), ('स', 5), ('ो', 1), ('य', 8), ('म', 6), ('ल', 5), ('व', 11), ('ट', 3), ('े', 17), ('क', 14), ('ी', 6), ('ए', 1), ('च', 2), ('ु', 2), ('आ', 8), ('भ', 3), ('ं', 3), ('ळ', 1), ('ण', 5), ('श', 6), ('ष', 4), ('ः', 1), ('ै', 1), ('ू', 1), ('ज', 2), ('थ', 1), ('अ', 3), ('ग', 1), ('ध', 1), ('ठ', 1), ('ड', 1)])


# Max Jugaad

In [17]:
tokens_indic = trivial_tokenize_indic(char_sequence_1)

tokens_indic = pd.Series(tokens_indic)

word_counts = tokens_indic.value_counts()
print(word_counts)

आ     8
क     7
र     6
प     5
प्    5
     ..
री    1
त     1
रा    1
ही    1
हो    1
Length: 67, dtype: int64


# Converting Devanagri Text to Char array

In [144]:
text = 'हरिदास होय मला वाटते की हा एक महत्त्वाचा मुद्दा आहे की भारतीय संदर्भामुळे आपण विशेषतः आमच्या शैक्षणिक प्रक्रियेद्वारे प्रवेश करू शकता जिथे प्रवेश परीक्षा असते आणि जी आपल्याला विभागांमध्ये ठेवते आणि काही आपण त्या विशिष्ट क्रियाकलापांवर अडकले आहात असे कसे वाटते'
clean_text = pp.clean_text(text)

token_dict = pp.tokenize_characters(clean_text)
text_seq = pp.text_to_sequence(clean_text, token_dict)

print('\n Text:\n',clean_text,'---->' ,len((clean_text).split()))
print('\n Character array:\n',pp.text2characters(clean_text),'---->' ,len(pp.text2characters(clean_text).split()))
print('\n Sequence array:\n',text_seq[0],'---->',len(text_seq[0]))

sent
Total Unique Tokens (Characters): 66
Records converted: 1

 Text:
 हरिदास होय मला वाटते की हा एक महत्त्वाचा मुद्दा आहे की भारतीय संदर्भामुळे आपण विशेषतः आमच्या शैक्षणिक प्रक्रियेद्वारे प्रवेश करू शकता जिथे प्रवेश परीक्षा असते आणि जी आपल्याला विभागांमध्ये ठेवते आणि काही आपण त्या विशिष्ट क्रियाकलापांवर अडकले आहात असे कसे वाटते ----> 41

 Character array:
 ह रि दा स हो य म ला वा ट ते की हा ए क म ह त् त् वा चा मु द् दा आ हे की भा र ती य सं द र् भा मु ळे आ प ण वि शे ष तः आ म च् या शै क् ष णि क प् र क् रि ये द् वा रे प् र वे श क रू श क ता जि थे प् र वे श प री क् षा अ स ते आ णि जी आ प ल् या ला वि भा गां म ध् ये ठे व ते आ णि का ही आ प ण त् या वि शि ष् ट क् रि या क ला पां व र अ ड क ले आ हा त अ से क से वा ट ते  ----> 135

 Sequence array:
 [19, 52, 41, 24, 29, 49, 56, 18, 46, 5, 9, 45, 6, 43, 36, 56, 19, 7, 7, 46, 64, 42, 62, 41, 58, 27, 45, 47, 31, 65, 49, 35, 57, 30, 47, 42, 32, 58, 2, 16, 44, 4, 38, 48, 58, 56, 17, 40, 28, 11, 38, 61, 36, 14, 31, 11, 52, 8, 62, 46, 55, 14, 31, 54, 60, 36

In [120]:
#------------------------Trial on original dataset-------------------------------------#

df = pd.read_csv('../Technodifacation/Data/training_data_marathi.csv')
x_train = df['text'].apply(lambda x : pp.clean_text(x)).tolist()
len(x_train)

41997

In [122]:
# Two step conversion

token_dict = pp.tokenize_characters(x_train)
x_train_tokenized = pp.text_to_sequence(x_train, token_dict)

Doc
Total Unique Tokens (Characters): 790
Total records:  41997
Records converted:  41997


In [152]:
import random
i = random.randint(0,len(x_train))
print('Text: \n {} \nNum Words: {} \n\n Sequence: \n {}\n Sequence len: {}'.format(x_train[i],len(x_train[i].split()),
                                                                    x_train_tokenized[i],len(x_train_tokenized[i])))

Text: 
 किमान  10  12 आइस क्रीम तेथे आहेत आणि अशा प्रकारे व्यवस्था केली जाते की किमान  8 ते  10 लॉली एका ठिकाणी ठेवल्या जाऊ शकतात 
Num Words: 24 

 Sequence: 
 [449, 354, 500, 723, 283, 723, 25, 486, 309, 119, 17, 420, 673, 108, 506, 486, 446, 91, 486, 683, 393, 52, 600, 141, 311, 382, 209, 60, 113, 568, 665, 389, 99, 607, 108, 159, 449, 354, 500, 407, 108, 723, 283, 249, 99, 352, 311, 174, 311, 104, 340, 113, 322, 153, 607, 271, 583, 631, 669, 91]
 Sequence len: 60
